In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

## running the model

In [4]:
from Regard import init
args = init(verbose=1)
print('Parameters=', args)


ModuleNotFoundError: No module named 'torch'

In [ ]:
from Regard import ML
ml = ML(args)

import os
import torch

path = '_Regard.pt'
ml.model.load_state_dict(torch.load(path))
print('Loading file', path)


from torchvision import transforms
t = transforms.Compose([
    transforms.CenterCrop(args.crop),
    transforms.Resize(args.size),
    transforms.ToTensor(),
    #transforms.Normalize(mean=self.IMAGENET_MEAN, std=self.IMAGENET_STD),
    transforms.Normalize(mean=[args.mean]*3, std=[args.std]*3),
    ])



In [ ]:
ml.dataset.dataset.classes

# dynamical test

In [ ]:
from LeCheapEyeTracker.EyeTrackerServer import Server
#from LeCheapEyeTracker.EyeTrackerClient import Client


def classify(image, t):
    from PIL import Image


    image = Image.fromarray(image.astype('uint8'), 'RGB')
    data = t(image)
    data.unsqueeze_(0)
    output = ml.model(data)
    
    pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
    return pred



def dyn_test(N_frame=64, startup_time=1., interframe_time=0., waiting_time=.0, display=False):
    try:
        import time
        time.sleep(startup_time)
        et = Server()
        t0 =et.clock()
        print ('Time', t0)
        frame = et.cam.grab()
        ready = False
        while ready is False and ( et.clock()-t0 <5 ):
            try:
                img_face, res, t0 = et.process_frame(frame, et.clock())
                H, W, three = img_face.shape
                ready = False
            except:
                pass
        
        for i in range(N_frame):            
            # TODO: presentation of stimulus
            time.sleep(interframe_time)

            t0 = time.time()
            if True:#try:
                frame = et.cam.grab()
                img_face, res, t0_ = et.process_frame(frame, et.clock())
                img_face = img_face[:, :, ::-1]

                pred = classify(img_face, t)
                if display:
                    plt.imshow(img_face)
                    plt.show() 
                print('Prediction =', ml.dataset.dataset.classes[pred])
                print('Elapsed time =', '%0.3f' % ((time.time()-t0)*1000), 'ms')
            #except:
            #    pass
            time.sleep(waiting_time)
            
    finally:
        et.close()
        
dyn_test(display=True)        

In [ ]:
et = Server()

In [ ]:
frame = et.cam.grab()
#        
#        H, W, three = img_face.shape
        
plt.imshow(frame)

img_face, res, t0 = et.process_frame(frame, et.clock())

In [ ]:
et.close()